In [70]:
#!/venv/bin/python
from hume import HumeBatchClient
from hume.models.config import FaceConfig
from hume.models.config import ProsodyConfig
import pandas as pd
import numpy as np
import json 

client = HumeBatchClient("GB5GR8utyBkDmLHy5Kh7mtAGGebtMK5R9nDVhn7p5u7FMcT2")
urls = ["https://tmpfiles.org/dl/2979437/untitled.mp3"]
configs = [ProsodyConfig(identify_speakers = True)]
job = client.submit_job(urls, configs)

print(job)
print("Running...")

job.await_complete()
job.download_predictions("predictions.json")
print("Predictions downloaded to predictions.json")

job.download_artifacts("artifacts.zip")
print("Artifacts downloaded to artifacts.zip")

with open('/Users/gill/Documents/GitHub/calhacks/backend/predictions.json', 'r') as f:
    # Load the contents of the file into a variable
    data = json.load(f)

# Print the contents of the variable
print(data)


Job(id="0eb953ca-7694-4432-882f-20ebbd63f80c")
Running...
Predictions downloaded to predictions.json
Artifacts downloaded to artifacts.zip
[{'source': {'type': 'url', 'url': 'https://tmpfiles.org/dl/2979437/untitled.mp3'}, 'results': {'predictions': [{'file': 'untitled.mp3', 'file_type': 'audio', 'models': {'prosody': {'metadata': {'confidence': 0.9868164, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': 'Hello?', 'time': {'begin': 0.07924729999999996, 'end': 0.79838706}, 'confidence': 0.9970703, 'speaker_confidence': None, 'emotions': [{'name': 'Admiration', 'score': 0.03272364288568497}, {'name': 'Adoration', 'score': 0.03242945671081543}, {'name': 'Aesthetic Appreciation', 'score': 0.029946791008114815}, {'name': 'Amusement', 'score': 0.11723615974187851}, {'name': 'Anger', 'score': 0.04068536311388016}, {'name': 'Anxiety', 'score': 0.039799097925424576}, {'name': 'Awe', 'score': 0.025780988857150078}, {'name': 'Awkwardness', 'score': 0.

In [71]:
# Extract predictions
predictions = data[0]['results']['predictions'][0]['models']['prosody']['grouped_predictions'][0]['predictions']

# Flatten to get just the predicted text
texts = []
for pred in predictions:
    texts.append(pred['text'])

df = pd.DataFrame({'text': texts})

print(df)

                                                text
0                                             Hello?
1  I'm Marian. Yeah. Today, I had a really, reall...
2  so much and So depressed, but I'm so happy at ...
3                                      my die again.


In [72]:
flattened_data = []
for item in data:
    for prediction in item['results']['predictions'][0]['models']['prosody']['grouped_predictions']:
        for text_prediction in prediction['predictions']:
            for emotion in text_prediction['emotions']:
                flattened_data.append({
                    'Text': text_prediction['text'],
                    'Time_Begin': text_prediction['time']['begin'],
                    'Time_End': text_prediction['time']['end'],
                    'Emotion_Name': emotion['name'],
                    'Emotion_Score': emotion['score']
                })

# Create a Pandas DataFrame
df = pd.DataFrame(flattened_data)

# Display the DataFrame
print(df)

              Text  Time_Begin   Time_End            Emotion_Name  \
0           Hello?    0.079247   0.798387              Admiration   
1           Hello?    0.079247   0.798387               Adoration   
2           Hello?    0.079247   0.798387  Aesthetic Appreciation   
3           Hello?    0.079247   0.798387               Amusement   
4           Hello?    0.079247   0.798387                   Anger   
..             ...         ...        ...                     ...   
187  my die again.   13.083043  14.199130     Surprise (negative)   
188  my die again.   13.083043  14.199130     Surprise (positive)   
189  my die again.   13.083043  14.199130                Sympathy   
190  my die again.   13.083043  14.199130               Tiredness   
191  my die again.   13.083043  14.199130                 Triumph   

     Emotion_Score  
0         0.032724  
1         0.032429  
2         0.029947  
3         0.117236  
4         0.040685  
..             ...  
187       0.028955  
188

In [102]:
grouped_df = df.groupby('Text')

# Display the grouped DataFrame
groups = []
texts = []
for name, group in grouped_df:
    print(f"Text: {name}")
    group = group.sort_values(by="Emotion_Score",ascending=False) 
    group = group[group["Emotion_Score"].apply(float) > .10]
    groups.append(group)
    texts.append(name)

Text: Hello?
Text: I'm Marian. Yeah. Today, I had a really, really, really terrible day. I wanna cry
Text: my die again.
Text: so much and So depressed, but I'm so happy at the same time. Yay. I might kill myself. I'm


In [103]:
groups
td

[      Text  Time_Begin  Time_End Emotion_Name  Emotion_Score
 9   Hello?    0.079247  0.798387     Calmness       0.200696
 31  Hello?    0.079247  0.798387     Interest       0.151885
 8   Hello?    0.079247  0.798387      Boredom       0.141842
 3   Hello?    0.079247  0.798387    Amusement       0.117236
 7   Hello?    0.079247  0.798387  Awkwardness       0.114563
 32  Hello?    0.079247  0.798387          Joy       0.106937,
                                                  Text  Time_Begin  Time_End  \
 88  I'm Marian. Yeah. Today, I had a really, reall...    1.276022  7.021613   
 79  I'm Marian. Yeah. Today, I had a really, reall...    1.276022  7.021613   
 77  I'm Marian. Yeah. Today, I had a really, reall...    1.276022  7.021613   
 66  I'm Marian. Yeah. Today, I had a really, reall...    1.276022  7.021613   
 57  I'm Marian. Yeah. Today, I had a really, reall...    1.276022  7.021613   
 68  I'm Marian. Yeah. Today, I had a really, reall...    1.276022  7.021613   
 
   